# Find duplicate questions on StackOverflow by their embeddings

In this assignment you will learn how to calculate a similarity for pieces of text. Using this approach you will know how to find duplicate questions from [StackOverflow](https://stackoverflow.com).

### Libraries

In this task you will you will need the following libraries:
- [StarSpace](https://github.com/facebookresearch/StarSpace) — a general-purpose model for efficient learning of entity embeddings from Facebook
- [Gensim](https://radimrehurek.com/gensim/) — a tool for solving various NLP-related tasks (topic modeling, text representation, ...)
- [Numpy](http://www.numpy.org) — a package for scientific computing.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — a tool for data mining and data analysis.
- [Nltk](http://www.nltk.org) — a platform to work with human language data.

### Data

The following cell will download all data required for this assignment into the folder `week3/data`.

In [1]:
import sys
sys.path.append("..")
from common.download_utils import download_week3_resources

download_week3_resources()

### Grading
We will create a grader instace below and use it to collect your answers. Note that these outputs will be stored locally inside grader and will be uploaded to platform only after running submiting function in the last part of this assignment. If you want to make partial submission, you can run that cell any time you want.

In [2]:
from grader import Grader

In [3]:
grader = Grader()

## Word embedding

To solve the problem, you will use two different models of embeddings:

 - [Pre-trained word vectors](https://code.google.com/archive/p/word2vec/) from Google which were trained on a part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases. You need to download it by following this [link](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing).
 - Representations using StarSpace on StackOverflow data sample. You will need to train them from scratch.

It's always easier to start with pre-trained embeddings. Unpack the pre-trained Goggle's vectors and upload them using the function [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) from gensim library with the parameter *binary=True*. If the size of the embeddings is larger than the avaliable memory, you could load only a part of the embeddings by defining the parameter *limit* (recommended: 500000).

In [7]:
import gensim
from gensim.models import KeyedVectors

In [13]:
wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin",binary=True)

In [25]:
#wv_embeddings['big']

most_similar = wv_embeddings.most_similar(positive=['woman', 'king'], negative=['man'])
most_similar_to_given = wv_embeddings.most_similar_to_given('music', ['water', 'sound', 'backpack', 'mouse'])

most_similar
most_similar_to_given

'sound'

### How to work with Google's word2vec embeddings?

Once you have loaded the representations, make sure you can access them. First, you can check if the loaded embeddings contain a word:
    
    'word' in wv_embeddings
    
Second, to get the corresponding embedding you can use the square brackets:

    wv_embeddings['word']
 
### Checking that the embeddings are correct 
 
To prevent any errors during the first stage, we can check that the loaded embeddings are correct. You can call the function *check_embeddings*, implemented below, which runs 3 tests:
1. Find the most similar word for provided "positive" and "negative" words.
2. Find which word from the given list doesn’t go with the others.
3. Find the most similar word for the provided one.

In the right case the function will return the string *These embeddings look good*. Othervise, you need to validate the previous steps.

In [14]:
def check_embeddings(embeddings):
    error_text = "Something wrong with your embeddings ('%s test isn't correct)."
    most_similar = embeddings.most_similar(positive=['woman', 'king'], negative=['man'])
    if len(most_similar) < 1 or most_similar[0][0] != 'queen':
        return error_text % "Most similar"

    doesnt_match = embeddings.doesnt_match(['breakfast', 'cereal', 'dinner', 'lunch'])
    if doesnt_match != 'cereal':
        return error_text % "Doesn't match"
    
    most_similar_to_given = embeddings.most_similar_to_given('music', ['water', 'sound', 'backpack', 'mouse'])
    if most_similar_to_given != 'sound':
        return error_text % "Most similar to given"
    
    return "These embeddings look good."

In [15]:
print(check_embeddings(wv_embeddings))

These embeddings look good.


## From word to text embeddings

**Task 1 (Question2Vec).** Usually, we have word-based embeddings, but for the task we need to create a representation for the whole question. It could be done in different ways. In our case we will use a **mean** of all word vectors in the question. Now you need to implement the function *question_to_vec*, which calculates the question representation described above. This function should work with the input text as is without any preprocessing.

Note that there could be words without the corresponding embeddings. In this case, you can just skip these words and don't take them into account during calculating the result. If the question doesn't contain any known word with embedding, the function should return a zero vector.

In [29]:
import numpy as np

In [150]:
def question_to_vec(question, embeddings, dim=300):
    """
        question: a string
        embeddings: dict where the key is a word and a value is its' embedding
        dim: size of the representation

        result: vector representation for the question
    """
    vect=np.zeros(300)
    count=0
    if question is not '':
        #print("second")
        for word in question.split():
            #print(word,len(question.split()))
            if word in embeddings:
                vect=vect+embeddings[word]
             
            else:
                count=count + 1
                #print(count,len(question))
                if (count==len(question.split())):
                    return np.zeros(300)
        #print(vect)        
        return vect/(len(question.split())-count)       
    else:
        #print("first")
        return np.zeros(300)
   
            
        
        
    
            
    
            
        
        
    ######################################
    ######### YOUR CODE HERE #############
    ######################################

To check the basic correctness of your implementation, run the function *question_to_vec_tests*.

In [84]:
def question_to_vec_tests():
    if (np.zeros(300) != question_to_vec('', wv_embeddings)).any():
        return "You need to return zero vector for empty question."
    if (np.zeros(300) != question_to_vec('thereisnosuchword', wv_embeddings)).any():
        return "You need to return zero vector for the question, which consists only unknown words."
    if (wv_embeddings['word'] != question_to_vec('word', wv_embeddings)).any():
        return "You need to check the corectness of your function."
    if ((wv_embeddings['I'] + wv_embeddings['am']) / 2 != question_to_vec('I am', wv_embeddings)).any():
        return "Your function should calculate a mean of word vectors."
    if (wv_embeddings['word'] != question_to_vec('thereisnosuchword word', wv_embeddings)).any():
        return "You should not consider words which embeddings are unknown."
    return "Basic tests are passed."

In [85]:
print(question_to_vec_tests())

thereisnosuchword 1
word 1
[ 3.59375000e-01  4.15039062e-02  9.03320312e-02  5.46875000e-02
 -1.47460938e-01  4.76074219e-02 -8.49609375e-02 -2.04101562e-01
  3.10546875e-01 -1.05590820e-02 -6.15234375e-02 -1.55273438e-01
 -1.52343750e-01  8.54492188e-02 -2.70996094e-02  3.84765625e-01
  4.78515625e-02  2.58789062e-02  4.49218750e-02 -2.79296875e-01
  9.09423828e-03  4.08203125e-01  2.40234375e-01 -3.06640625e-01
 -1.80664062e-01  4.73632812e-02 -2.63671875e-01  9.08203125e-02
  1.37695312e-01 -7.20977783e-04  2.67333984e-02  1.92382812e-01
 -2.29492188e-02  9.70458984e-03 -7.37304688e-02  4.29687500e-01
 -7.93457031e-03  1.06445312e-01  2.80761719e-02 -2.29492188e-01
 -1.91650391e-02 -2.36816406e-02  3.51562500e-02  1.71875000e-01
 -1.12304688e-01  6.25000000e-02 -1.69921875e-01  1.29882812e-01
 -1.54296875e-01  1.58203125e-01 -7.76367188e-02  1.78710938e-01
 -1.72851562e-01  9.96093750e-02  3.94531250e-01  6.44531250e-02
 -6.83593750e-02 -3.18359375e-01  5.95703125e-02 -1.02539062e-0

 -0.15527344 -0.13879395 -0.20996094 -0.06097412  0.15991211 -0.01953125]
thereisnosuchword 2
word 2
[ 3.59375000e-01  4.15039062e-02  9.03320312e-02  5.46875000e-02
 -1.47460938e-01  4.76074219e-02 -8.49609375e-02 -2.04101562e-01
  3.10546875e-01 -1.05590820e-02 -6.15234375e-02 -1.55273438e-01
 -1.52343750e-01  8.54492188e-02 -2.70996094e-02  3.84765625e-01
  4.78515625e-02  2.58789062e-02  4.49218750e-02 -2.79296875e-01
  9.09423828e-03  4.08203125e-01  2.40234375e-01 -3.06640625e-01
 -1.80664062e-01  4.73632812e-02 -2.63671875e-01  9.08203125e-02
  1.37695312e-01 -7.20977783e-04  2.67333984e-02  1.92382812e-01
 -2.29492188e-02  9.70458984e-03 -7.37304688e-02  4.29687500e-01
 -7.93457031e-03  1.06445312e-01  2.80761719e-02 -2.29492188e-01
 -1.91650391e-02 -2.36816406e-02  3.51562500e-02  1.71875000e-01
 -1.12304688e-01  6.25000000e-02 -1.69921875e-01  1.29882812e-01
 -1.54296875e-01  1.58203125e-01 -7.76367188e-02  1.78710938e-01
 -1.72851562e-01  9.96093750e-02  3.94531250e-01  6.44

You can submit embeddings for the questions from the file *test_embeddings.tsv* to earn the points. In this task you don't need to transform the text of a question somehow.

In [79]:
from util import array_to_string

In [87]:
question2vec_result = []
for question in open('data/test_embeddings.tsv'):
    question = question.strip()
    answer = question_to_vec(question, wv_embeddings)
    question2vec_result = np.append(question2vec_result, answer)

grader.submit_tag('Question2Vec', array_to_string(question2vec_result))

question2vec_result

Play 13
Framework 13
1.2.5 13
: 13
OutOfMemoryError 13
occured 13
: 13
Java 13
heap 13
space 13
in 13
play 13
framework 13
[ 0.17364502 -0.25854492  0.414505    0.76733398  0.21875    -0.65612793
  0.12084961  0.01049805  0.12756348  0.77973175 -0.85957336 -0.11669922
  0.01147461 -0.96142578 -0.83740234 -0.37890625  0.10269165  0.66772461
 -0.02685547 -0.54458618  0.20489502  0.89501953  0.04577637  1.01756287
  0.52752686  0.29602051  0.29310608  1.19165039  0.46745682 -0.76171875
 -1.14050293 -0.4251709   0.06359863 -1.00028992  0.22576904 -0.52246094
  0.05004883  0.30838013  0.72112274  0.95733643  0.7414856  -0.45697021
  0.30206299  0.9163208   0.21269226 -0.96429443 -1.09997559 -1.58740234
 -0.07885742  1.02636719  0.5133667  -0.0090332  -0.24230957 -0.72094727
 -0.36413574  0.24590302  0.02124023 -1.37182617 -0.20483398 -0.83447266
 -0.22924805  0.29272461 -0.28259277 -0.52978516 -0.1685791   0.30883789
  0.20761871  0.7019043  -0.21166992  0.11125946  0.90600586 -1.14526367
 

 -0.07580566 -0.38085938 -0.29931641  0.24462891  0.20623779  0.41607666]
Formula 17
to 17
auto 17
change 17
all 17
calendar 17
hours 17
if 17
I 17
change 17
1 17
to 17
keep 17
the 17
same 17
total 17
hours 17
[ 7.03125000e-02 -6.45904541e-01  3.92089844e-01  1.41479492e+00
 -1.17578125e+00 -5.17028809e-01 -2.56958008e-02 -1.11602783e+00
  1.52185059e+00  9.62348938e-01 -6.28112793e-01 -6.31225586e-01
 -6.47827148e-01  6.92260742e-01 -1.80224609e+00  1.07336426e+00
  8.89831543e-01  7.64244080e-01  1.29229736e+00 -7.01637268e-01
 -7.22900391e-01  1.03836060e+00 -6.45263672e-01 -6.93443298e-01
  6.84053421e-01  2.22116089e+00 -1.30554199e+00  1.12167358e+00
  1.34031677e+00  6.15936279e-01 -1.59887695e+00 -3.59130859e-01
 -4.50859070e-01 -5.74645996e-02  7.66082764e-01 -5.11230469e-01
 -1.60034180e-01  6.22314453e-01 -7.20458984e-01  1.73785400e+00
  1.84814453e+00 -3.56628418e-01  1.33673096e+00  4.53063965e-01
 -1.12536621e+00 -8.10165405e-01 -5.87829590e-01  9.64111328e-01
  5.872802

  0.31445312  0.03405762  0.37445068 -0.36645508 -0.4096756   0.34362793]
How 13
to 13
read 13
response 13
from 13
angular 13
resource 13
$save() 13
and 13
also 13
keeping 13
original 13
data 13
[ 0.26948547 -0.08299923  0.4536438   0.26345444 -0.60827637  0.17456055
  0.29003906 -0.36132812  0.83879089  0.73510742 -0.43356323 -0.97509766
 -0.74960327  0.56864929 -0.89031982 -0.07348633 -0.10662842  0.92456055
 -0.62023926 -1.15203857 -0.11474609  0.57873535 -0.36505127 -0.3059082
 -0.29348755 -0.1149292  -0.63464355  0.96801758  0.02319336 -0.93334961
  0.53167725 -0.75415039 -0.37646484 -0.39361572  0.1060791  -0.05407715
  0.57751465  0.10675812  0.13092041  0.57617188  0.29309082  0.42642212
  0.60412598  0.44812393 -0.33361816 -0.63876343 -0.82434082 -0.00195312
  0.13891602  0.17886353 -0.31958008 -1.08032227 -0.49041748 -1.14941406
  0.93357849  1.10510254 -1.11328125 -0.29699707  0.40330505  0.5078125
 -0.20595551 -0.45184326 -1.3503418  -0.79650879 -0.54205322  0.17982483
 -0.

  0.32537842  0.234375    0.34387207 -0.68060303  0.02941895  0.33740234]
View 10
what 10
methods/functions 10
return 10
a 10
specific 10
object 10
in 10
Visual 10
Studio 10
[ 0.70211792 -0.28009033 -0.29394531  0.13037109 -1.06298828  0.53222656
  0.328125   -0.86135578  0.46020508  0.35778809 -0.20849609 -0.18847656
  0.67480469  0.11083984 -0.94360352  0.18908691  0.48913574  0.71072388
 -0.49298096 -0.69685364 -0.48913574  0.53820801 -1.18786621  0.50189209
 -0.40350342  0.06640625  0.46266174  0.44287109  1.11169434 -0.58679199
 -0.74768066 -0.09863281 -1.2322998  -0.8125     -0.25770569 -0.93896484
  0.60571289 -0.21911621 -0.32299805  0.43408966  0.5680542   0.19830322
  0.43283081  0.26593018 -0.02685547 -0.43579102 -0.30196381 -0.54187012
 -0.07592773  0.68554688 -0.70874023  0.06091309 -0.48364258 -0.37226868
 -0.34796143  0.40511322 -0.3538208  -0.19567871  0.1887207  -0.44335938
  0.33215332  0.25927734 -0.73242188 -0.64364624 -0.76647949 -0.30297852
  0.37158203  0.5994873

 -1.73583984e-01 -1.13134766e+00  1.21582031e-01  3.72528076e-01]
Installing 15
an 15
Aurelia 15
web 15
application 15
on 15
a 15
web 15
server 15
other 15
than 15
ones 15
for 15
the 15
turorial 15
[ 0.46112061 -0.52294922 -0.12553406  0.43896484 -0.67871094  0.95739746
  0.71615601 -0.18676758  1.35679626  0.69372559  0.09667969 -1.05981445
 -0.78900146 -0.17456055  0.49499512  1.08401489  0.08984375  0.83203125
 -0.44940186  0.04345703 -0.05908203  0.52893066 -0.36032104  0.66329193
  0.40234375 -0.56689453 -0.84399414  1.90435791 -0.07478333 -0.86425781
 -0.19525146 -0.49246979 -0.65264893 -1.19851685  0.55609131 -0.72479248
 -0.07843018  0.30535889  1.11291504  0.17150879  0.5871582  -0.03990936
  0.78100586  0.7093811   0.42796326 -0.94352722 -0.25323486  0.39122009
  0.02160645  0.6003418  -0.22328186  0.38299561  0.33355713 -1.50387573
 -0.27978516  0.90045166 -0.34802246  0.38256836  0.27639771 -0.09985352
 -0.66775513 -0.20962524 -1.45556641 -0.75372314 -1.08422852  0.45869446

...

 -2.03613281e-01 -1.30926514e+00 -6.59423828e-01  5.22460938e-02]
android: 9
removing 9
item 9
from 9
listview 9
is 9
not 9
working 9
properly 9
[-2.82623291e-01  8.63122940e-02  1.35803223e-01  4.34875488e-01
 -1.02624512e+00  3.25561523e-01  4.84375000e-01 -3.17138672e-01
  1.97998047e-01  9.55810547e-02 -2.08801270e-01 -9.77050781e-01
 -6.03027344e-02  1.69311523e-01 -1.02490234e+00  6.14135742e-01
  1.00097656e-01  5.45104980e-01  2.23388672e-02  2.49633789e-02
  2.17773438e-01 -1.01074219e-01 -4.27490234e-01 -2.32055664e-01
  1.54327393e-01 -5.05371094e-02 -4.77294922e-01  7.18872070e-01
 -1.16271973e-01  6.34765625e-03 -2.54638672e-01  6.65283203e-02
 -1.02172852e-01  1.72080994e-02  5.49560547e-01 -2.91503906e-01
  1.81152344e-01 -2.05688477e-01 -1.79138184e-01 -1.74072266e-01
  4.82788086e-01  7.73193359e-01  7.21313477e-01 -4.64538574e-01
 -6.59660339e-01 -4.85321045e-01 -2.83203125e-02  2.32910156e-01
 -3.32763672e-01  6.89086914e-02  3.69224548e-02 -8.53881836e-02
 -2.246704

 -8.97216797e-01  1.66992188e-01  1.58996582e-01 -2.42919922e-02]
Information 10
to 10
store 10
in 10
exception 10
object: 10
providing 10
strings 10
in 10
exception 10
[ 0.07821655 -0.18530273  0.14111328  0.23950195  0.02793884  0.21630859
  0.33166504 -0.87518311  0.45483398  0.73486328 -1.30078125 -0.41555405
 -0.30847168 -0.14465332 -1.23594666 -0.24487305  0.41241455  0.37261963
 -0.48632812 -0.78662109  0.38922119  0.6953125  -0.61709595 -0.05957031
  0.73620605  0.27685547  0.03091431  0.66015625 -0.08056641  0.33544922
  0.0769043  -0.70861816 -0.3157959   0.41821289 -0.29632568  0.29614258
 -0.22412109 -0.27069092  0.21765137  0.47033691  0.76226807  0.56195068
 -0.51898193 -0.00585938  0.38427734 -1.13818359 -0.40527344  0.31787109
 -0.34716797  0.27978516 -0.60449219  0.04806519 -0.59436035 -0.0769043
  0.13476562  0.31260681 -0.62695312 -0.20043945  0.1105957  -0.54437256
 -0.13195801 -0.15612793 -0.46228027 -1.70507812 -0.68261719  0.50439453
 -0.90625     0.94165039 -0.1

 -4.37988281e-01  3.51623535e-01  1.15966797e-01  3.13476562e-01]
Trouble 11
working 11
on 11
Java 11
app 11
between 11
Windows 11
and 11
Snow 11
Leopard 11
machines 11
[ 1.16394043e+00 -1.06591797e+00 -1.30938721e+00  8.04626465e-01
 -9.53735352e-01  1.11669922e+00  3.44665527e-01  6.50512695e-01
  7.70385742e-01  9.88281250e-01 -7.00271606e-01  5.18310547e-01
  3.52416992e-01 -1.20886230e+00 -5.75820923e-01 -2.27294922e-01
  6.33117676e-01  8.43383789e-01 -3.12568665e-01 -1.21475220e+00
 -7.32421875e-02  1.85302734e-01  1.79199219e-01  5.50872803e-01
  1.06665039e+00  1.00170898e+00 -1.44335938e+00  1.78875732e+00
  5.59326172e-01 -8.71154785e-01 -8.16955566e-01 -6.92665100e-01
  1.33483887e-01 -6.95556641e-01 -5.21240234e-02 -6.04980469e-01
 -7.69775391e-01  1.26159668e-01  7.11509705e-01  7.37915039e-01
  1.09234619e+00 -4.90417480e-01  4.07257080e-02  3.53027344e-01
 -6.25076294e-01 -4.57817078e-01 -1.16650391e+00 -6.55639648e-01
 -4.97314453e-01  7.27905273e-01  9.20104980e-02 -2

 -0.31494141 -0.29602051  0.03649902  0.44946289  0.25920105  0.08325195]
MySql 9
5.7 9
installer 9
fails 9
to 9
detect 9
VS 9
2013 9
redistributable 9
[ 3.97949219e-01 -7.55004883e-02  2.41210938e-01 -2.66357422e-01
 -7.55371094e-01  7.11669922e-01  4.30908203e-01  4.34082031e-01
  3.56140137e-01  4.88281250e-04 -9.88769531e-01 -4.37255859e-01
 -4.97924805e-01 -2.61993408e-01 -4.34326172e-01  8.76953125e-01
  5.22460938e-01  1.08709717e+00 -7.29980469e-01 -6.33728027e-01
 -2.02636719e-01  8.15307617e-01  3.99322510e-01  2.34863281e-01
 -2.49755859e-01 -8.59375000e-01 -6.81884766e-01  1.28515625e+00
 -5.96191406e-01 -7.68554688e-01 -6.32019043e-01 -8.08105469e-02
 -5.67504883e-01 -9.93041992e-01 -5.33691406e-01 -4.96093750e-01
 -4.82055664e-01  2.61474609e-01  6.95800781e-02 -1.88217163e-02
  5.89965820e-01 -1.73492432e-01  1.30102539e+00 -1.07666016e-01
 -5.04028320e-01 -8.61328125e-01 -5.73638916e-01 -6.52832031e-01
 -1.55517578e-01 -9.75341797e-02 -6.57348633e-01 -2.49023438e-01
  4

array([ 0.01929389, -0.02872721,  0.04605611, ..., -0.11884562,
       -0.01780192,  0.01442464])

Now we have a method to create a representation of any sentence and we are ready for the first evaluation. So, let's check how well our solution (Google's vectors + *question_to_vec*) will work.

## Evaluation of text similarity

We can imagine that if we use good embeddings, the cosine similarity between the duplicate sentences should be less than for the random ones. Overall, for each pair of duplicate sentences we can generate *R* random negative examples and find out the position of the correct duplicate.  

For example, we have the question *"Exceptions What really happens"* and we are sure that another question *"How does the catch keyword determine the type of exception that was thrown"* is a duplicate. But our model doesn't know it and tries to find out the best option also among questions like *"How Can I Make These Links Rotate in PHP"*, *"NSLog array description not memory address"* and *"PECL_HTTP not recognised php ubuntu"*. The goal of the model is to rank all these 4 questions (1 *positive* and *R* = 3 *negative*) in the way that the correct one is in the first place.

However, it is unnatural to count on that the best candidate will be always in the first place. So let us consider the place of the best candidate in the sorted list of candidates and formulate a metric based on it. We can fix some *K* — a reasonalble number of top-ranked elements and *N* — a number of queries (size of the sample).

### Hits@K

The first simple metric will be a number of correct hits for some *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)]$$

where $q_i$ is the i-th query, $dup_i$ is its duplicate, $topK(q_i)$ is the top K elements of the ranked sentences provided by our model and the operation $[dup_i \in topK(q_i)]$ equals 1 if the condition is true and 0 otherwise (more details about this operation could be found [here](https://en.wikipedia.org/wiki/Iverson_bracket)).


### DCG@K
The second one is a simplified [DCG metric](https://en.wikipedia.org/wiki/Discounted_cumulative_gain):

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K] $$

where $rank_{dup_i}$ is a position of the duplicate in the sorted list of the nearest sentences for the query $q_i$. According to this metric, the model gets a higher reward for a higher position of the correct answer. If the answer does not appear in topK at all, the reward is zero. 

### Evaluation examples

Let's calculate the described metrics for the toy example introduced above. In this case $N$ = 1 and the correct candidate for $q_1$ is *"How does the catch keyword determine the type of exception that was thrown"*. Consider the following ranking of the candidates:
1. *"How Can I Make These Links Rotate in PHP"*
2. *"How does the catch keyword determine the type of exception that was thrown"*
3. *"NSLog array description not memory address"*
4. *"PECL_HTTP not recognised php ubuntu"*

Using the ranking above, calculate *Hits@K* metric for *K = 1, 2, 4*: 
 
- [K = 1] $\text{Hits@1} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top1(q_i)] = [dup_1 \in top1(q_1)] = 0$ because the correct answer doesn't appear in the *top1* list.
- [K = 2] $\text{Hits@2} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top2(q_i)] = [dup_1 \in top2(q_1)] = 1$ because $rank_{dup_1} = 2$.
- [K = 4] $\text{Hits@4} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top4(q_i)] = [dup_1 \in top4(q_1)] = 1$

Using the ranking above, calculate *DCG@K* metric for *K = 1, 2, 4*:

- [K = 1] $\text{DCG@1} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = \frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = 0$ because the correct answer doesn't appear in the top1 list.
- [K = 2] $\text{DCG@2} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 2] = \frac{1}{\log_2{3}}$, because $rank_{dup_1} = 2$.
- [K = 4] $\text{DCG@4} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 4] = \frac{1}{\log_2{3}}$.


**Tasks 2 and 3 (HitsCount and DCGScore).** Implement the functions *hits_count* and *dcg_score* as described above. Each function has two arguments: *dup_ranks* and *k*. *dup_ranks* is a list which contains *values of ranks* of duplicates. For example, *dup_ranks* is *[2]* for the example provided above.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
        k: number of top-ranked elements (k in Hits@k metric)

        result: return Hits@k value for current ranking
    """
    ######################################
    ######### YOUR CODE HERE #############
    ######################################

In [102]:
answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
candidates_ranking = [["How Can I Make These Links Rotate in PHP", "NSLog array description not memory address",
                           "How does the catch keyword determine the type of exception that was thrown",
                           
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
#dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
#dup_ranks
print(candidates_ranking[0])
candidates_ranking[0].index(answers[0])

['How Can I Make These Links Rotate in PHP', 'NSLog array description not memory address', 'How does the catch keyword determine the type of exception that was thrown', 'PECL_HTTP not recognised php ubuntu']


2

Test your code on the tiny examples:

In [ ]:
def test_hits():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1, 1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]
    
    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."

    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [ ]:
print(test_hits())

In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
        k: number of top-ranked elements (k in DCG@k metric)

        result: return DCG@k value for current ranking
    """
    ######################################
    ######### YOUR CODE HERE #############
    ######################################

In [ ]:
def test_dcg():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]

    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, (1 + (1 / (np.log2(3)))) / 2]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."
        
    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [ ]:
print(test_dcg())

Submit results of the functions *hits_count* and *dcg_score* for the following examples to earn the points.

In [ ]:
test_examples = [
    [1],
    [1, 2],
    [2, 1],
    [1, 2, 3],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [9, 5, 4, 2, 8, 10, 7, 6, 1, 3],
    [4, 3, 5, 1, 9, 10, 7, 8, 2, 6],
    [5, 1, 7, 6, 2, 3, 8, 9, 10, 4],
    [6, 3, 1, 4, 7, 2, 9, 8, 10, 5],
    [10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
]

In [ ]:
hits_results = []
for example in test_examples:
    for k in range(len(example)):
        hits_results.append(hits_count(example, k + 1))
grader.submit_tag('HitsCount', array_to_string(hits_results))

In [ ]:
dcg_results = []
for example in test_examples:
    for k in range(len(example)):
        dcg_results.append(dcg_score(example, k + 1))
grader.submit_tag('DCGScore', array_to_string(dcg_results))

##  First solution: pre-trained embeddings

We will work with predefined train, validation and test corpora. All the files are tab-separated, but have a different format:
 - *train* corpus contains similar sentences at the same row.
 - *validation* corpus contains the following columns: *question*, *similar question*, *negative example 1*, *negative example 2*, ... 
 - *test* corpus contains the following columns: *question*, *example 1*, *example 2*, ...

Validation corpus will be used for the intermediate validation of models. The test data will be necessary for submitting the quality of your model in the system.

Now you should upload *validation* corpus to evaluate current solution.

In [105]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [126]:
train = read_corpus('data/train.tsv')

In [110]:
validation = read_corpus('data/validation.tsv')

In [139]:
len(validation[1])
#train

1001

In [104]:
from sklearn.metrics.pairwise import cosine_similarity

We will use cosine distance to rank candidate questions which you need to implement in the function *rank_candidates*. The function should return a sorted list of pairs *(initial position in candidates list, candidate)*. Index of some pair corresponds to its rank (the first is the best). For example, if the list of candidates was *[a, b, c]* and the most similar is *c*, then *a* and *b*, the function should return a list *[(2, c), (0, a), (1, b)]*.

Pay attention, if you use the function *cosine_similarity* from *sklearn.metrics.pairwise* to calculate similarity because it works in a different way: most similar objects has greatest similarity. It's preferable to use a vectorized version of *cosine_similarity* function. Try to compute similarity at once and not use list comprehension. It should speed up your computations significantly.

In [427]:
def rank_candidates(question, candidates, embeddings, dim=300):
    """
        question: a string
        candidates: a list of strings (candidates) which we want to rank
        embeddings: some embeddings
        dim: dimension of the current embeddings
        
        result: a list of pairs (initial position in the list, question)
    """
    score=[]
    index=[]
    return_list=[]
    question_vec=question_to_vec(question,embeddings).reshape(1,-1)
    #print(question_vec)
    #print(question_vec.shape)
    for i in range(len(candidates)):
        #print(candidate)
        candidate_vec=question_to_vec(candidates[i],embeddings).reshape(1,-1)
        #print(candidate_vec.shape)
        res=cosine_similarity(question_vec,candidate_vec)
        
        
        
        #print(res[0][0])
        score.append([(res[0][0]),i,candidates[i]])
        #print(score)
        
    sorted_score=sorted(score,reverse=True)
    #sorted_score.remove([0][0])

    #print(sorted_score[0][0])
    #print(len(sorted_score))
    for i in  range (len(sorted_score)):
        index.append(sorted_score[i][1])
        #print(index)   
        
   # print(sorted_score)   
        
    for j in index:
            return_list.append([j,candidates[j]])
        
   
    return return_list  
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################

Test your code on the tiny examples:

In [428]:
def test_rank_candidates():
    questions = ['converting string to list', 'Sending array via Ajax fails']
    candidates = [['Convert Google results object (pure js) to Python object', 
                   'C# create cookie from string and send it',
                   'How to use jQuery AJAX for an outside domain?'], 
                  ['Getting all list items of an unordered list in PHP', 
                   'WPF- How to update the changes in list item of a list', 
                   'select2 not displaying search results']]
    results = [[(1, 'C# create cookie from string and send it'), 
                (0, 'Convert Google results object (pure js) to Python object'), 
                (2, 'How to use jQuery AJAX for an outside domain?')],
               [(0, 'Getting all list items of an unordered list in PHP'), 
                (2, 'select2 not displaying search results'), 
                (1, 'WPF- How to update the changes in list item of a list')]]
    for question, q_candidates, result in zip(questions, candidates, results):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, 300)
        print(ranks)
        if not np.all(ranks == result):
            return "Check the function."
    return "Basic tests are passed."

In [431]:
print(test_rank_candidates())

[[1, 'C# create cookie from string and send it'], [0, 'Convert Google results object (pure js) to Python object'], [2, 'How to use jQuery AJAX for an outside domain?']]
Check the function.


Now we can test the quality of the current approach. Run the next two cells to get the results. Pay attention that calculation of similarity between vectors takes time and this calculation is computed approximately in 10 minutes.

In [432]:
wv_ranking = []
for line in validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [435]:
validation[0]

['How to print a binary heap tree without recursion?',
 'How do you best convert a recursive function to an iterative one?',
 'How can i use ng-model with directive in angular js',
 'flash: drawing and erasing',
 'toggle react component using hide show classname',
 'Use a usercontrol from another project to current webpage',
 '~ Paths resolved differently after upgrading to ASP.NET 4',
 'Materialize datepicker - Rendering when an icon is clicked',
 'Creating PyPi package - Could not find a version that satisfies the requirement iso8601',
 'How can I analyze a confusion matrix?',
 'How do I declare a C array in Swift?',
 'Using rand() when flipping a coin and rolling a die',
 'Handling a JSON field with a special character in its name in Java',
 'React Native select row on ListView when push it',
 "Get 'creation_time' of video using ffmpeg and regex",
 'Does row exist and multiple where',
 "How to specify a classifier in a gradle dependency's dependency?",
 'Using $unwind on multiple do

In [434]:

ranks

[[0,
  'Replace conditional with polymorphism - nice in theory but not practical'],
 [522,
  'Different output for the same input using scipy optimize with L-BFGS-B algorithm'],
 [90,
  'Factor Clojure code setting many different fields in a Java object using a parameter map bound to a var or local'],
 [607,
  'What are the downsides of specifying synonyms directly in config instead of using synonyms_path'],
 [802,
  'Need to create li with list of different links using php explode method'],
 [349,
  'Passing a simple IEnumerable to view and using foreach to loop through returns a blank screen?'],
 [425,
  'Assert that certain parameterized vectors will throw an exception in JUnit?'],
 [900,
  'Which documentation should I use to develop & test makefile based applications with Yocto?'],
 [241,
  'How do you override an existing html form to use jquery to post the form?'],
 [189,
  'Difference between using and not using pipe in Export-Csv in Powershell'],
 [866,
  'I cannot use alphanu

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

If you did all the steps correctly, you should be frustrated by the received results. Let's try to understand why the quality is so low. First of all, when you work with some data it is necessary to have an idea how the data looks like. Print several questions from the data:

In [134]:
for line in validation[:3]:
    q, *examples = line
    print(q, *examples[:3])
    print('------------')
    print(q)
    print('------------')

How to print a binary heap tree without recursion? How do you best convert a recursive function to an iterative one? How can i use ng-model with directive in angular js flash: drawing and erasing
------------
How to print a binary heap tree without recursion?
------------
How to start PhoneStateListener programmatically? PhoneStateListener and service Java cast object[] to model WCF and What does this mean?
------------
How to start PhoneStateListener programmatically?
------------
jQuery: Show a div2 when mousenter over div1 is over when hover on div1 depenting on if it is on div2 or not it should act differently How to run selenium in google app engine/cloud? Python Comparing two lists of strings for similarities
------------
jQuery: Show a div2 when mousenter over div1 is over
------------


As you can see, we deal with the raw data. It means that we have many punctuation marks, special characters and unlowercased letters. In our case, it could lead to the situation where we can't find some embeddings, e.g. for the word "grid?". 

To solve this problem you should use the functions *text_prepare* from the previous assignments to prepare the data.

In [ ]:
from util import text_prepare

Now transform all the questions from the validation set:

In [ ]:
prepared_validation = []
for line in validation:
    ######### YOUR CODE HERE #############

Let's evaluate the approach again after the preparation:

In [ ]:
wv_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_prepared_ranking, k), 
                                              k, hits_count(wv_prepared_ranking, k)))

Now, prepare also train and test data, because you will need it in the future:

In [ ]:
def prepare_file(in_, out_):
    out = open(out_, 'w')
    for line in open(in_, encoding='utf8'):
        line = line.strip().split('\t')
        new_line = [text_prepare(q) for q in line]
        print(*new_line, sep='\t', file=out)
    out.close()

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

**Task 4 (W2VTokenizedRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates to earn the points. The calculations should take about 3-5 minutes. Pay attention that the function *rank_candidates* returns a ranking, while in this case you should find a position in this ranking. Ranks should start with 1.

In [ ]:
from util import matrix_to_string

In [ ]:
w2v_ranks_results = []
prepared_test_data = ######### YOUR CODE HERE #############
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, wv_embeddings, 300)
    ranked_candidates = [r[0] for r in ranks]
    w2v_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('W2VTokenizedRanks', matrix_to_string(w2v_ranks_results))

## Advanced solution: StarSpace embeddings

Now you are ready to train your own word embeddings! In particular, you need to train embeddings specially for our task of duplicates detection. Unfortunately, StarSpace cannot be run on Windows and we recommend to use provided
[docker container](https://github.com/hse-aml/natural-language-processing/blob/master/Docker-tutorial.md) or other alternatives. Don't delete results of this task because you will need it in the final project.

### How it works and what's the main difference with word2vec?
The main point in this section is that StarSpace can be trained specifically for some tasks. In contrast to word2vec model, which tries to train similar embeddings for words in similar contexts, StarSpace uses embeddings for the whole sentence (just as a sum of embeddings of words and phrases). Despite the fact that in both cases we get word embeddings as a result of the training, StarSpace embeddings are trained using some supervised data, e.g. a set of similar sentence pairs, and thus they can better suit the task.

In our case, StarSpace should use two types of sentence pairs for training: "positive" and "negative". "Positive" examples are extracted from the train sample (duplicates, high similarity) and the "negative" examples are generated randomly (low similarity assumed). 

### How to choose the best params for the model?
Normally, you would start with some default choice and then run extensive experiments to compare different strategies. However, we have some recommendations ready for you to save your time:
- Be careful with choosing the suitable training mode. In this task we want to explore texts similarity which corresponds to *trainMode = 3*.
- Use adagrad optimization (parameter *adagrad = true*).
- Set the length of phrase equal to 1 (parameter *ngrams*), because we need embeddings only for words.
- Don't use a large number of *epochs* (we think that 5 should be enough).
- Try dimension *dim* equal to 100.
- To compare embeddings usually *cosine* *similarity* is used.
- Set *minCount* greater than 1 (for example, 2) if you don't want to get embeddings for extremely rare words.
- Parameter *verbose = true* could show you the progress of the training process.
- Set parameter *fileFormat* equals *labelDoc*.
- Parameter *negSearchLimit* is responsible for a number of negative examples which is used during the training. We think that 10 will be enought for this task.
- To increase a speed of training we recommend to set *learning rate* to 0.05.

Train StarSpace embeddings for unigrams on the train dataset. You don't need to change the format of the input data. Just don't forget to use prepared version of the training data. 

If you follow the instruction, the training process will take about 1 hour.

In [ ]:
######### TRAINING HAPPENING HERE #############

And now we can compare the new embeddings with the previous ones. You can find trained word vectors in the file *[model_file_name].tsv*. Upload the embeddings from StarSpace into a dict. 

In [ ]:
starspace_embeddings = ######### YOUR CODE HERE #############

In [ ]:
ss_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ss_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ss_prepared_ranking, k), 
                                               k, hits_count(ss_prepared_ranking, k)))

Due to training for the particular task with the supervised data, you should expect to obtain a higher quality than for the previous approach. In additiion, despite the fact that StarSpace's trained vectors have a smaller dimension than word2vec's, it provides better results in this task.

**Task 5 (StarSpaceRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates for trained representation.

In [ ]:
starspace_ranks_results = []
prepared_test_data = ######### YOUR CODE HERE #############
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ranked_candidates = [r[0] for r in ranks]
    starspace_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('StarSpaceRanks', matrix_to_string(starspace_ranks_results))

### Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. <b>Note:</b> Token expires 30 minutes after generation.

In [ ]:
STUDENT_EMAIL = # EMAIL 
STUDENT_TOKEN = # TOKEN 
grader.status()

If you want to submit these answers, run cell below

In [ ]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)